<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Notes" data-toc-modified-id="Notes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Notes</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Plot-the-results-of-the-analysis" data-toc-modified-id="Plot-the-results-of-the-analysis-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot the results of the analysis</a></span><ul class="toc-item"><li><span><a href="#Analysis-of-06012023" data-toc-modified-id="Analysis-of-06012023-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Analysis of 06012023</a></span></li><li><span><a href="#Analyze-a-specific-case" data-toc-modified-id="Analyze-a-specific-case-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Analyze a specific case</a></span></li></ul></li><li><span><a href="#Analyze-specific-cases" data-toc-modified-id="Analyze-specific-cases-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Analyze specific cases</a></span></li><li><span><a href="#How-do-things-change-when-we-only-change-eps?" data-toc-modified-id="How-do-things-change-when-we-only-change-eps?-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>How do things change when we only change eps?</a></span></li></ul></div>

# Description

We want to: 
- build a good set of functions to compute the size control
- run a sensitivity analysis of the size control as a function of the mass scaling and the relative degradation rate

The parameters that we expect to have a significant impact are: 
- the type of transition
- the rate of transition `k`
- the degradation ratio `epsilon`

The way we do it is: 

In a script:
- Run simulations for a single cell (I don't think it's useful to use populations in this case)
- Save simulation results
- Compute slopes for the different metrics we plot under `plot_size_control`

Here
- Analyze the results

# Notes

Current observations are that: 
- `k_trans` needs to be very high to have a perfectly cycling system. However, even for moderate values, you have a little spread which translates into a slope, but that might be completely meaningless
- `k_trans` infuses sufficient stochasticity in the system for short simulations to be wrongly informative in terms of the slopes. We need multiple samples
- `transition_th` seems to have some influence on the slope character... To be confirmed. 

Things we can check: 
- CV of the mass at birth: this needs to be sufficiently large to measure a real size control - but cannot be too large either otherwise you have abnormally long transition phase. 
- introduce more noise elsewhere. e.g. (i) G2 duration: that will change where we land and therefore will disrupt the MG1. (ii) asymmetric division: this will also disrupt where we land
- the time in G1 as a function of `k_trans` (should not impact it too much) and `transition_th` (should impact it a lot!

# Load

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import os

import pickle

In [ ]:
from RBmodel import plot_utils, load_utils

In [ ]:
def _viz_slope(stats, df_):
    """
    visualize the slope of the size control plot
    """
    
    plt.figure()

    x = stats['birth'] - np.mean(stats['birth'])
    y1 = stats['delta'][0] - np.mean(stats['delta'][0])
    slope_ = (df_['slopes_G1_delta']) 
    eps = df_["epsilon"]
    y2 = slope_* x

    plt.scatter(
            x, y1, s=2
        )
    plt.grid()
    plt.plot(
        x, y2
    )


    plt.title(f"Viz of the estimated slope: {np.round(slope_, decimals=2)}; eps={eps}")
    plt.xlabel("M at birth")
    plt.ylabel("M_G1S - M birth")
    return

# Plot the results of the analysis

The cells below load the results file `results.csv` and plot one figure for each transition type. The figure is saved locally.

**NOTE** This is valid for the first analysis (that was iterating through 4 parameters). Not for the latest analysis. 

In [ ]:
EXP_DIR = "/Users/lucasfuentes/RB_model/data/size_control"

df_exp = pd.read_csv(os.path.join(EXP_DIR, "results.csv"), index_col = 0)

In [ ]:
plot_utils.plot_size_control_slope_analysis(df_exp, "/Users/lucasfuentes/RB_model/figs")

## Analysis of 06012023

Here we realize that the two bigger factors might be k and transition threshold in determining the size control capacity. 

Of course, it might depend on other parameters. 

The function `plot_utils.plot_size_control_slope_analysis()` is not adapted to this analysis, so we will do it here for now. 

In [ ]:
EXP_DIR = "/Users/lucasfuentes/RB_model/data/size_control/060623"

df_exp = pd.read_csv(os.path.join(EXP_DIR, "results.csv"), index_col = 0)

In [ ]:
uniq_k = df_exp['k_trans'].unique()

In [ ]:
uniq_e = df_exp["epsilon"].unique()

In [ ]:
uniq_d = df_exp["delta"].unique()

In [ ]:
uniq_d

In [ ]:
d = .8

In [ ]:
fig, ax = plt.subplots(len(uniq_e), 2, figsize=(10, 3*len(uniq_e)))

for (i, eps) in enumerate(uniq_e):
    df_exp_ = df_exp.loc[(df_exp['epsilon']==eps) & (df_exp['delta']==d)]

    for k in uniq_k: 
        df_ = df_exp_.loc[df_exp_['k_trans']==k]
        ax[i, 0].plot(df_['transition_th'], df_['slopes_G1_delta'], 'o--',label=f"k={k}")
        ax[i, 1].plot(df_['transition_th'], df_['CV_M_birth'] * 100, 'o--',label=f"k={k}")

#     ax[i, 0].legend()
    ax[i, 0].grid()
    ax[i, 0].set_xlabel("transition th")
    ax[i, 0].set_ylabel("Slope")

    ax[i, 1].legend()
    ax[i, 1].grid()
    ax[i, 1].set_xlabel("transition th")
    ax[i, 1].set_ylabel("CV_M_birth [%]")
    
    ax[i, 0].set_title(f"eps = {eps}")
    ax[i, 1].set_title(f"eps = {eps}")
    
fig.tight_layout()

What do we observe?
- CV is very much set by `k` and `th`. `eps` does not impact the result in any way
- When `eps` is low, the `th` has an impact in how much size control there is. if `eps` is high, the `th` does not really change anything. This seems to be the case regardless of the value of `k`

The sensitivity of the slope increases when `delta` increases. The more linear it is, the more `th` will impact the slope at low values of `k`. 

**Note**: you also cannot neglect the role of `alpha` in the system. This directly tunes the synthesis rate, and its relative value compared to `gamma` is important.

## Analyze a specific case

In [ ]:
uniq_k

In [ ]:
uniq_e

In [ ]:
df_exp["transition_th"].unique()

In [ ]:
tr = "RBc"
k_trans = .05
eps = 0.1
th = 2.

In [ ]:
df_ = df_exp.loc[
    (df_exp["transition"]==tr) & (df_exp["delta"]==d) & (df_exp["epsilon"]==eps) & (df_exp["k_trans"]==k_trans) & (df_exp["transition_th"]==th)
]
df_ = df_.loc[df_.index[0]]

fnm = df_["stats_file_path"]

stats = load_utils.load_size_control_analysis_results(fnm)

plot_utils.plot_size_control(stats=stats)

TODO: plot G1 duration as a function of several parameters (e.g. transition th)

In [ ]:
plt.scatter(df_exp["slopes_G1_delta"], df_exp["slopes_G2_delta"], alpha=.3)
plt.grid()
plt.xlabel("G1 size control")
plt.ylabel("Full cell cycle size control")

# Analyze specific cases

In [ ]:
# choose the parameters for which to plot

tr = "RBc"
delta = .5
k_trans = 1.

In [ ]:
eps = .5

In [ ]:
df_ = df_exp.loc[
    (df_exp["transition"]==tr) & (df_exp["delta"]==delta) & (df_exp["epsilon"]==eps) & (df_exp["k_trans"]==k_trans)
]
df_ = df_.loc[df_.index[0]]

fnm = df_["stats_file_path"]

stats = load_utils.load_size_control_analysis_results(fnm)

In [ ]:
plot_utils.plot_size_control(stats=stats)

In [ ]:
_viz_slope(stats, df_)

# How do things change when we only change eps? 

We expect a change in size control as eps changes... can we visualize that? 

In [ ]:
df_eps = df_exp.loc[
    (df_exp["transition"]==tr) & (df_exp["delta"]==delta) & (df_exp["k_trans"]==k_trans)
]

In [ ]:
for idx in df_eps.index:
    
    df_crt = df_eps.loc[idx]
    fnm = df_crt["stats_file_path"]

    stats = load_utils.load_size_control_analysis_results(fnm)
    
    _viz_slope(stats, df_crt)

Plot the time spent in G1 as a function of eps!